<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/4_06%ED%8C%8C%EC%9D%B4%EC%8D%AC(danawa%EC%82%AC%EC%9D%B4%ED%8A%B8_%ED%81%AC%EB%A1%A4%EB%A7%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
[문제189] 다나와사이트에서 가격정보를 수집하세요.( 애플 노트북 )
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from pandas import Series, DataFrame
import pandas as pd
import time

# 다나와 사이트 접속
url = 'http://www.danawa.com/'
driver = webdriver.Chrome('c:/data/chromedriver.exe')
driver.get(url)

# 카테고리에서 노트북 elemnent로 들어가기
driver.find_element(By.XPATH,'//*[@id="sectionLayer"]/li[2]/a').click() # 컴퓨터,노트북,조립pc 카테고리 클릭

#driver.find_element(By.XPATH,'//*[@id="categoryHoverLayer11"]/div[1]/ul/li[4]/a/span').click() # 노트북 클릭 (현재 시행안되고 오류나는 부분)
action = ActionChains(driver)
action.move_to_element(driver.find_element(By.XPATH,'//*[@id="categoryHoverLayer11"]/div[1]/ul/li[4]/a/span')).perform() # 노트북 카테고리에 마우스 올리기

driver.find_element(By.XPATH,'//*[@id="categoryUnit96"]/ul/li[1]/a').click() # 노트북 전체 클릭

# 상세검색 제조사별 -> 더보기 버튼 클릭 -> apple체크박스 클릭
driver.find_element(By.CLASS_NAME,'btn_spec_view.btn_view_more').click() # 더보기 버튼
driver.find_element(By.ID,'searchMaker1452').click() # apple체크박스

# 첫 페이지에서 광고를 뺀 애플 제품들의 title, 스펙 ,메모리, ssd, 가격(+버튼에 나와있는 회사와 가격까지) 추출하여 dataframe에 담기
html = driver.page_source # 현재 자동화로 켜져있는 웹page source 추출
driver.quit() # 자동화 웹 종료

soup = BeautifulSoup(html,'html.parser') # parsing
data = soup.select_one('div.main_prodlist > ul.product_list > li.prod_item') # 첫 번째 제품 div

# title 제목
title = data.select_one('div.prod_info > p.prod_name > a').text.strip()

# 스펙
spec = data.select_one('div.prod_info > dl.prod_spec_set > dd > div.spec_list').text.strip()

# 메모리, ssd
m_s = data.select_one('div.prod_pricelist > ul > li > div.over_preview > p').text.strip() #.split(',') ssd랑 메모리랑 나눌 때


for i in data.select('div.prod_pricelist > ul > li > div.over_preview > p'):
    print(i.text.strip())

# 가격(2개 이상)
data.select('div.prod_pricelist > ul > li > p.price_sect > a > strong')

# 더보기 회사, 가격 # CSS선택자로 다시 하기
driver.find_element(By.CLASS_NAME,'ico.i_more').click()
driver.find_element(By.XPATH,'//*[@id="productInfoDetail_14766710"]/p[2]/span/button').click()
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

    # 회사명
data.select_one('div.prod_pricelist > ul > li > p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_logo > img')['alt']

for i in data.select('div.prod_pricelist > ul > li > p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_logo > img'):#['alt']
     print(i['alt'])

    # 가격
data.select('div.prod_pricelist > ul > li > p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_price > span') #.text.replace('원','')

#DataFrame에 담기
apple_notebook_df = DataFrame(columns=['title','spec','memory/ssd','sales_company','price'])

sales_company = []
price = []
for i in data.select('div.prod_pricelist > ul > li > p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_logo > img'):#['alt']
     sales_company.append(i['alt'])

for i in data.select('div.prod_pricelist > ul > li > p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_price > span'):
    price.append(i.text.replace('원',''))

for i in range(0,len(sales_company)):
    apple_notebook_df = apple_notebook_df.append({'title':title,'spec' : spec,'memory/ssd':m_s, 'sales_company':sales_company[i], 'price':price[i]},ignore_index=True)

apple_notebook_df # 한개 노트북 1개memory만 저장해보는 테스트

◈문제 해결방안 생각하기(풀어나갈 구조 작성)
> 첫 번째 노트북의 가격 더보기를 클릭하면서 html source 를 추출
> title과 spec, memory/ssd, 더보기의 회사, 가격은 리스트형식으로 저장
> 그 다음 다시 더보기를 클릭하고 똑같은 작업을 반복 ( 가격더보기가 존재하는 p속성 수만큼)
> 그 다음 노트북도 똑같이 반복 ~ 그 페이지의 노트북 개수만큼
> 2페이지로 넘어가서 다시 같은 작업 반복

◈문제 해결방안 생각하기(풀어나갈 구조 작성)2
> 더보기 버튼을 클릭해야지 더보기 안에있는 HTML SOURCE를 가져올 수 있다
> 우선 현재 페이지의 모든 더보기 버튼을 클릭2번한다(클릭한 상태에서 다른 더보기 클릭 안됨)
> pagesource를 추출하고 데이터를 뽑아낸다. 
> 데이터 뽑아내는 작업이 끝나면 다음페이지로 넘어간다. 
> 똑같이 작업한다.
-------------------------------여러개의 memory 가격들 추출하기 2번방법-------------------------------
# 다나와 사이트 접속
url = 'http://www.danawa.com/'
driver = webdriver.Chrome('c:/data/chromedriver.exe')
driver.get(url)

# 카테고리에서 노트북 elemnent로 들어가기
driver.find_element(By.XPATH,'//*[@id="sectionLayer"]/li[2]/a').click() # 컴퓨터,노트북,조립pc 카테고리 클릭

#driver.find_element(By.XPATH,'//*[@id="categoryHoverLayer11"]/div[1]/ul/li[4]/a/span').click() # 노트북 클릭 (현재 시행안되고 오류나는 부분)
action = ActionChains(driver)
action.move_to_element(driver.find_element(By.XPATH,'//*[@id="categoryHoverLayer11"]/div[1]/ul/li[4]/a/span')).perform() # 노트북 카테고리에 마우스 올리기

driver.find_element(By.XPATH,'//*[@id="categoryUnit96"]/ul/li[1]/a').click() # 노트북 전체 클릭

# 상세검색 제조사별 -> 더보기 버튼 클릭 -> apple체크박스 클릭
driver.find_element(By.CLASS_NAME,'btn_spec_view.btn_view_more').click() # 더보기 버튼
driver.find_element(By.ID,'searchMaker1452').click() # apple체크박스

# 가격 더보기 첫번째 버튼(test)
driver.find_element(By.CSS_SELECTOR,'div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info > div.prod_pricelist > ul > li > p.price_sect > span').click()

# 전체 더 보기 버튼 클릭2번씩하기
li = driver.find_element(By.CSS_SELECTOR,'div.main_prodlist > ul.product_list') 

for j in li.find_elements_by_tag_name('li.prod_item'):
    
    # 노트북 당 가지고 있는 더보기 버튼들을 다 클릭(일시품절은 넘어가기[continue])
    the = j.find_element(By.CSS_SELECTOR,'div.prod_main_info > div.prod_pricelist > ul ')
    
    for i in the.find_elements_by_tag_name('li') :
        try:
            i.find_element(By.CSS_SELECTOR,'p.price_sect > span.more_btn_wrap > button').click()
            i.find_element(By.CSS_SELECTOR,'p.price_sect > span.more_btn_wrap > button').click()
            time.sleep(2)
        except:
            continue # try문은 '일시품절'이면 오류가 나기때문에 사용
                    
# page.source 추출
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')


# title
soup.select_one('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info > div.prod_info > p.prod_name > a').text.strip()
# spec
soup.select_one('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info > div.prod_info > dl > dd > div.spec_list').text.strip()
# memory/ssd
soup.select_one('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info > div.prod_pricelist > ul > li > div.over_preview').text.strip()
# 회사명
soup.select_one('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info > div.prod_pricelist > ul > li >p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_logo > img')['alt']
# 가격
soup.select_one('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info > div.prod_pricelist > ul > li >p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_price > span').text

# 한 페이지 최종코드
apple_notebook_df = DataFrame(columns=['title','spec','memory/ssd','sales_company','price'])
sales_company = []
price = []
li = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
len(li)
for i in li:
    # title
    title = i.select_one('div.prod_main_info > div.prod_info > p.prod_name > a').text.strip()
    # spec
    spec = i.select_one('div.prod_main_info > div.prod_info > dl > dd > div.spec_list').text.strip()
    
    for j in i.select('div.prod_main_info > div.prod_pricelist > ul > li > div.over_preview'): # memory/ssd의 개수만큼 반복
        m_s = j.text.strip()
        sales_company = []
        price = []
        for k in i.select('div.prod_main_info > div.prod_pricelist > ul > li >p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_logo > img'):
            sales_company.append(k['alt'])
        for z in i.select('div.prod_main_info > div.prod_pricelist > ul > li >p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_price > span'):
            price.append(z.text)
            
    apple_notebook_df = apple_notebook_df.append({'title':title,'spec':spec,'memory/ssd':m_s,'sales_company':sales_company,'price':price},ignore_index=True)
apple_notebook_df['title']
apple_notebook_df['spec']
apple_notebook_df['memory/ssd']
apple_notebook_df['sales_company']
apple_notebook_df['price']

# 최종
apple_notebook_df = DataFrame(columns=['title','spec','memory/ssd','sales_company','price'])

for k in range(1,4):
    driver.find_element(By.CSS_SELECTOR,'div.number_wrap >a:nth-of-type('+str(k)+')').click()
    time.sleep(2)
    li2 = driver.find_element(By.CSS_SELECTOR,'div.main_prodlist > ul.product_list') 
    for j in li2.find_elements_by_tag_name('li.prod_item'):
        
        # 노트북 당 가지고 있는 더보기 버튼들을 다 클릭(일시품절은 넘어가기[continue])
        the = j.find_element(By.CSS_SELECTOR,'div.prod_main_info > div.prod_pricelist > ul ')
        
        for i in the.find_elements_by_tag_name('li') :
            try:
                i.find_element(By.CSS_SELECTOR,'p.price_sect > span.more_btn_wrap > button').click()
                i.find_element(By.CSS_SELECTOR,'p.price_sect > span.more_btn_wrap > button').click()
                time.sleep(2)
            except:
                continue # try문은 '일시품절'이면 오류가 나기때문에 사용
                                    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    li = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
    
    for i in li:
        # title
        title = i.select_one('div.prod_main_info > div.prod_info > p.prod_name > a').text.strip()
        # spec
        spec = i.select_one('div.prod_main_info > div.prod_info > dl > dd > div.spec_list').text.strip()
        
        for j in i.select('div.prod_main_info > div.prod_pricelist > ul > li > div.over_preview'): # memory/ssd의 개수만큼 반복
            m_s = j.text.strip()
            sales_company = []
            price = []
            #  손봐야됨회사랑 가
            for k in i.select('div.prod_main_info > div.prod_pricelist > ul > li >p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_logo > img'):
                sales_company.append(k['alt'])
            for z in i.select('div.prod_main_info > div.prod_pricelist > ul > li >p.price_sect > span.more_btn_wrap > span.layer_price_more > span.lpm_wrap > a.lpm_price > span'):
                price.append(z.text)
                
            apple_notebook_df = apple_notebook_df.append({'title':title,'spec':spec,'memory/ssd':m_s,'sales_company':sales_company,'price':price},ignore_index=True)
            time.sleep(2)                
        
    
apple_notebook_df['title']
apple_notebook_df['spec']
apple_notebook_df['memory/ssd']
apple_notebook_df['price']
apple_notebook_df.to_csv('c:/data/danawa_apple_notebook_1.csv')
x = pd.read_csv('c:/data/danawa_apple_notebook_1.csv')

------------------------------------------------------
# 강사님의 뼈대 

# 상세검색 제조사별 -> 더보기 버튼 클릭 -> apple체크박스 클릭
driver.find_element(By.XPATH,'//*[@id="dlMaker_simple"]/dd/div[2]/button[1]').click()
driver.find_element(By.XPATH,'//*[@id="searchMaker1452"]').click()

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

# 제목과 스펙
li = soup.select('div.main_prodlist.main_prodlist_list > ul > li.prod_item.prod_layer')
prod_name = li[0].select_one('p.prod_name > a').text.strip()

prod_info = ''
for i in li[0].select('div.spec_list > a'):
    prod_info += i.text + ','

# 가격, memory/ssd
//*[@id="productInfoDetail_12660491"]/p[2]/span/button
//*[@id="productInfoDetail_14766710"]/p[2]/span/button

input = li[0].select('div.prod_pricelist > ul > li')
id_value=[]
for i in input:
    id_value.append(i.find('input',{"name":"compareValue"})['value'])

prod_dic = {}

driver.find_element(By.XPATH,'//*[@id="productInfoDetail_{}"]/p[2]/span/button'.format(id_value[0])).click()
html_2 = driver.page_source
soup_2 = BeautifulSoup(html_2,'html.parser')

tag1 = 'span#layer_price_more_{} > span.lpm_wrap' .format(id_value[0])
info = ''
for i in soup_2.select(tag1):
    info += i.select_one('a > img')['alt'] + '/' + i.select_one('a.lpm_price').text.strip() + ' '

tag2 = 'li#productInfoDetail_{} > div >p.memory_sect'.format(id_value[0])
prod_dic[soup_2.select_one(tag2).text.strip()] = info


driver.find_element(By.XPATH,'//*[@id="productInfoDetail_{}"]/p[2]/span/button'.format(id_value[1])).click()
html_2 = driver.page_source
soup_2 = BeautifulSoup(html_2,'html.parser')

tag1 = 'span#layer_price_more_{} > span.lpm_wrap' .format(id_value[1])
info = ''
for i in soup_2.select(tag1):
    info += i.select_one('a > img')['alt'] + '/' + i.select_one('a.lpm_price').text.strip() + ' '

tag2 = 'li#productInfoDetail_{} > div >p.memory_sect'.format(id_value[1])
prod_dic[soup_2.select_one(tag2).text.strip()] = info

df = DataFrame(columns=['제품명','제품정보','제품가격'])
df = df.append({'제품명':prod_name,'제품정보':prod_info,'제품가격':prod_dic},ignore_index=True)
df
driver.find_element(By.XPATH,'//*[@id="productInfoDetail_{}"]/p[2]/span/button'.format(id_value[0])).click()

driver.find_element(By.XPATH,'//*[@id="productInfoDetail_{}"]/p[2]/span/button'.format(id_value[1])).click()
driver.find_element(By.XPATH,'//*[@id="productInfoDetail_{}"]/p[2]/span/button'.format(id_value[1])).click()

//*[@id="productListArea"]/div[5]/div/div/a[1]
//*[@id="productListArea"]/div[5]/div/div/a[2]
//*[@id="productListArea"]/div[5]/div/div/a[3]
#페이지 변경
driver.find_element(By.XPATH,'//*[@id="productListArea"]/div[5]/div/div/a[2]'.format(id_value[0])).click()
driver.find_element(By.CSS_SELECTOR,'div.number_wrap >a:nth-of-type(3)').click()
driver.find_element(By.CSS_SELECTOR,'div.number_wrap >a:nth-of-type(2)').click()

####################################################
------------강사님 뼈대로 완성시키기
url = 'http://www.danawa.com/'
driver = webdriver.Chrome('c:/data/chromedriver.exe')
driver.get(url)

# 카테고리에서 노트북 elemnent로 들어가기
driver.find_element(By.XPATH,'//*[@id="sectionLayer"]/li[2]/a').click() # 컴퓨터,노트북,조립pc 카테고리 클릭

#driver.find_element(By.XPATH,'//*[@id="categoryHoverLayer11"]/div[1]/ul/li[4]/a/span').click() # 노트북 클릭 (현재 시행안되고 오류나는 부분)
action = ActionChains(driver)
action.move_to_element(driver.find_element(By.XPATH,'//*[@id="categoryHoverLayer11"]/div[1]/ul/li[4]/a/span')).perform() # 노트북 카테고리에 마우스 올리기

driver.find_element(By.XPATH,'//*[@id="categoryUnit96"]/ul/li[1]/a').click() # 노트북 전체 클릭

# 상세검색 제조사별 -> 더보기 버튼 클릭 -> apple체크박스 클릭
driver.find_element(By.XPATH,'//*[@id="dlMaker_simple"]/dd/div[2]/button[1]').click()
driver.find_element(By.XPATH,'//*[@id="searchMaker1452"]').click()

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

# 제목과 스펙
li = soup.select('div.main_prodlist.main_prodlist_list > ul > li.prod_item.prod_layer')
prod_name = li[0].select_one('p.prod_name > a').text.strip()

prod_info = ''
for i in li[0].select('div.spec_list > a'):
    prod_info += i.text + ','


    
